In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers biopython kaggle py-boost
print("✅ Đã cài đặt xong thư viện!")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.8 MB/s eta 0:00:00
✅ Đã cài đặt xong thư viện!


In [ ]:

import os
from google.colab import files

# Upload file kaggle.json
print("Vui lòng upload file kaggle.json của bạn:")
files.upload()

# Cấu hình Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Tải dữ liệu cuộc thi (Sẽ mất khoảng 1-2 phút)
print("⏳ Đang tải dữ liệu CAFA 6...")
!kaggle competitions download -c cafa-6-protein-function-prediction
!unzip -q cafa-6-protein-function-prediction.zip -d /content/cafa6_data
print("✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data")

Vui lòng upload file kaggle.json của bạn:


Saving kaggle.json to kaggle.json
⏳ Đang tải dữ liệu CAFA 6...
 67% 61.0M/91.3M [00:00<00:00, 637MB/s]
100% 91.3M/91.3M [00:00<00:00, 464MB/s]
✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data


In [ ]:
import numpy as np
import pandas as pd
import torch
import gc
import os
import sys
import joblib
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.decomposition import IncrementalPCA # Dùng cái này để ko tràn RAM thường

# --- IMPORT PY-BOOST ---
try:
    from py_boost import GradientBoosting
    from py_boost.multioutput.sketching import RandomProjectionSketch
    # from py_boost.gpu.losses import BCEWithNaNLoss
except ImportError:
    print("❌ Chưa cài Py-Boost! Hãy chạy: pip install py-boost")
    sys.exit()

# ============================================================================
# CONFIGURATION
# ============================================================================
CONFIG = {
    'EMBEDDINGS': {
        'ankh': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_test_FINAL_embeddings.npy',
        },
        'esm': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh/esm2_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh/esm2_test_FINAL_embeddings.npy',
        },
        'prot_t5': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/Prot_T5_Aligned/protT5_xl_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/Prot_T5_Aligned/protT5_xl_test_FINAL_embeddings.npy',
        },
    },

    'TRAIN_ID_PATH': '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_train_FINAL_ids.npy',
    'TEST_ID_PATH':  '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_test_FINAL_ids.npy',

    'SAVE_DIR': '/content/drive/MyDrive/CAFA6_Results/PyBoost_Run',
    'TRAIN_TERMS': '/content/cafa6_data/Train/train_terms.tsv',
    'IA_FILE': '/content/cafa6_data/IA.tsv',

    # --- GIẢM TẢI MEMORY Ở ĐÂY ---
    'TOP_K_LABELS': 5000,   # [QUAN TRỌNG] Giảm từ 5000 -> 2000 để cứu VRAM
    # 'LABEL_CHUNK_SIZE': 4000,

    'NTREES': 10000,
    'LR': 0.01,
    'MAX_DEPTH': 6,
    'ES': 300,
    'VERBOSE': 100,

    'MIN_CONFIDENCE': 0.01,
    'MAX_PREDS_PER_PROTEIN': 20,

    'SEED': 42,
    'DEVICE': 0
}

os.makedirs(CONFIG['SAVE_DIR'], exist_ok=True)
print(f"🚀 CAFA 6 - PY-BOOST LIGHTWEIGHT | GPU: {CONFIG['DEVICE']}")

# ============================================================================
# 1. DATA LOADING
# ============================================================================
def load_embeddings(embedding_config, mode='train'):
    print(f"\n⏳ Loading {mode} embeddings...")
    feature_list = []
    for name, paths in embedding_config.items():
        path = paths[mode]
        emb = np.load(path, mmap_mode='r')
        feature_list.append(emb)

    # Ghép nối
    X = np.concatenate([arr[:] for arr in feature_list], axis=1).astype(np.float32)
    print(f"   ✓ Raw Shape: {X.shape}")
    del feature_list, emb
    gc.collect()
    return X

# ============================================================================
# 2. PREPARE LABELS
# ============================================================================
print("\n[1/5] Processing Labels...")
train_ids = np.load(CONFIG['TRAIN_ID_PATH'])
df_terms = pd.read_csv(CONFIG['TRAIN_TERMS'], sep='\t', header=0, names=['EntryID', 'term', 'aspect'])
df_ia = pd.read_csv(CONFIG['IA_FILE'], sep='\t', names=['term', 'ia'])
ia_dict = dict(zip(df_ia['term'], df_ia['ia']))

term_counts = df_terms['term'].value_counts().reset_index()
term_counts.columns = ['term', 'freq']
term_counts['ia'] = term_counts['term'].map(ia_dict).fillna(0.0)
term_counts['score'] = term_counts['freq'] * term_counts['ia']

top_terms_df = term_counts.sort_values(by='score', ascending=False).head(CONFIG['TOP_K_LABELS'])
top_terms = top_terms_df['term'].tolist()
print(f"   ✓ Selected Top {len(top_terms)} terms (Reduced for VRAM safety).")

df_filtered = df_terms[df_terms['term'].isin(top_terms)]
id_to_terms = df_filtered.groupby('EntryID')['term'].apply(list).to_dict()

del df_terms, df_filtered, term_counts, top_terms_df
gc.collect()

print("   ⏳ Converting to Binary Matrix...")
mlb = MultiLabelBinarizer(classes=top_terms, sparse_output=True)
mlb.fit([top_terms])
y_labels_list = [id_to_terms.get(pid, []) for pid in train_ids]
Y_sparse = mlb.transform(y_labels_list)
Y = Y_sparse.astype(np.float32).toarray()

del y_labels_list, train_ids, Y_sparse
gc.collect()

# ============================================================================
# 3. LOAD FEATURES (X)
# ============================================================================
print("\n[2/4] Loading Features...")
X = load_embeddings(CONFIG['EMBEDDINGS'], mode='train')

# Split Train/Val
print("   ⏳ Splitting Train/Val...")
# Stratified split ở đây hơi khó vì multi-label, split random là ok
X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size=0.1, random_state=CONFIG['SEED']
)

# Giải phóng bản gốc X, Y để tiết kiệm RAM
del X, Y
gc.collect()

print(f"   Train shape: {X_train.shape}, {Y_train.shape}")
print(f"   Val shape:   {X_val.shape}, {Y_val.shape}")

# ============================================================================
# 4. TRAIN PY-BOOST
# ============================================================================
print("\n[3/5] Training Py-Boost Model...")
# Dọn sạch VRAM trước khi train
torch.cuda.empty_cache()

sketch = RandomProjectionSketch(3) # Giảm sketch xuống 3 để nhẹ hơn nữa

model = GradientBoosting(
    loss='bce',
    metric=None,
    ntrees=CONFIG['NTREES'],
    lr=CONFIG['LR'],
    max_depth=CONFIG['MAX_DEPTH'],
    min_data_in_leaf=20,
    colsample=0.8,
    subsample=0.8,
    lambda_l2=1,
    gd_steps=1,
    verbose=CONFIG['VERBOSE'],
    es=CONFIG['ES'],
    multioutput_sketch=sketch,
    use_hess=False,
    max_bin=64 # [MỚI] Giảm bin từ 256 -> 64 để tiết kiệm memory histogram
)

model.fit(X_train, Y_train, eval_sets=[{'X': X_val, 'y': Y_val}])

model_path = f"{CONFIG['SAVE_DIR']}/pyboost_model.pkl"
joblib.dump(model, model_path)
print(f"   🏆 Model saved to: {model_path}")

del X_train, Y_train, X_val, Y_val
gc.collect()

# ============================================================================
# 5. INFERENCE
# ============================================================================
print("\n[4/5] Inference on Test Set...")

# Load Test Features
# X_test = load_embeddings(CONFIG['EMBEDDINGS'], mode='test')

# test_ids = np.load(CONFIG['TEST_ID_PATH'])

# print("   🔮 Predicting...")
# preds = model.predict(X_test, batch_size=5000)

# print("   📝 Writing Submission...")
# submission_path = f"{CONFIG['SAVE_DIR']}/submission.tsv"
# n_predictions = 0

# with open(submission_path, 'w') as f:
#     for i, pid in enumerate(tqdm(test_ids)):
#         probs = preds[i]
#         top_k = CONFIG['MAX_PREDS_PER_PROTEIN']
#         ind = np.argpartition(probs, -top_k)[-top_k:]
#         ind = ind[np.argsort(probs[ind])][::-1]

#         for idx in ind:
#             score = probs[idx]
#             if score > CONFIG['MIN_CONFIDENCE']:
#                 term = top_terms[idx]
#                 f.write(f"{pid}\t{term}\t{score:.3f}\n")
#                 n_predictions += 1

# print(f"\n✅ DONE! Predictions: {n_predictions:,}")

🚀 CAFA 6 - PY-BOOST LIGHTWEIGHT | GPU: 0

[1/5] Processing Labels...
   ✓ Selected Top 5000 terms (Reduced for VRAM safety).
   ⏳ Converting to Binary Matrix...

[2/4] Loading Features...

⏳ Loading train embeddings...
   ✓ Raw Shape: (82404, 3072)
   ⏳ Splitting Train/Val...
   Train shape: (74163, 3072), (74163, 5000)
   Val shape:   (8241, 3072), (8241, 5000)

[3/5] Training Py-Boost Model...
[08:51:38] Stdout logging level is INFO.


INFO:py_boost.callbacks.callback:Stdout logging level is INFO.


[08:51:38] GDBT train starts. Max iter 10000, early stopping rounds 300


INFO:py_boost.callbacks.callback:GDBT train starts. Max iter 10000, early stopping rounds 300


[08:51:39] Iter 0; Sample 0, BCE = 0.004657233273928484; 


...

[09:15:51] Iter 2900; Sample 0, BCE = 0.003317166237058337; 


INFO:py_boost.callbacks.callback:Iter 2900; Sample 0, BCE = 0.003317166237058337; 


[09:16:00] Early stopping at iter 2919, best iter 2619, best_score 0.0033151899641220674


INFO:py_boost.callbacks.callback:Early stopping at iter 2919, best iter 2619, best_score 0.0033151899641220674


   🏆 Model saved to: /content/drive/MyDrive/CAFA6_Results/PyBoost_Run/pyboost_model.pkl

[4/5] Inference on Test Set...


In [ ]:
import numpy as np
import pandas as pd
import torch
import gc
import os
import sys
import joblib
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.decomposition import IncrementalPCA # Dùng cái này để ko tràn RAM thường

# --- IMPORT PY-BOOST ---
try:
    from py_boost import GradientBoosting
    from py_boost.multioutput.sketching import RandomProjectionSketch
    # from py_boost.gpu.losses import BCEWithNaNLoss
except ImportError:
    print("❌ Chưa cài Py-Boost! Hãy chạy: pip install py-boost")
    sys.exit()

# ============================================================================
# CONFIGURATION
# ============================================================================
CONFIG = {
    'EMBEDDINGS': {
        'ankh': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_test_FINAL_embeddings.npy',
        },
        'esm': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh/esm2_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh/esm2_test_FINAL_embeddings.npy',
        },
        'prot_t5': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/Prot_T5_Aligned/protT5_xl_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/Prot_T5_Aligned/protT5_xl_test_FINAL_embeddings.npy',
        },
    },

    'TRAIN_ID_PATH': '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_train_FINAL_ids.npy',
    'TEST_ID_PATH':  '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_test_FINAL_ids.npy',

    'SAVE_DIR': '/content/drive/MyDrive/CAFA6_Results/PyBoost_Run',
    'TRAIN_TERMS': '/content/cafa6_data/Train/train_terms.tsv',
    'IA_FILE': '/content/cafa6_data/IA.tsv',

    # --- GIẢM TẢI MEMORY Ở ĐÂY ---
    'TOP_K_LABELS': 5000,   # [QUAN TRỌNG] Giảm từ 5000 -> 2000 để cứu VRAM
    # 'LABEL_CHUNK_SIZE': 4000,

    'NTREES': 10000,
    'LR': 0.01,
    'MAX_DEPTH': 6,
    'ES': 200,
    'VERBOSE': 100,

    'MIN_CONFIDENCE': 0.01,
    'MAX_PREDS_PER_PROTEIN': 20,

    'SEED': 42,
    'DEVICE': 0
}

os.makedirs(CONFIG['SAVE_DIR'], exist_ok=True)
print(f"🚀 CAFA 6 - PY-BOOST LIGHTWEIGHT | GPU: {CONFIG['DEVICE']}")

# ============================================================================
# 1. DATA LOADING
# ============================================================================
def load_embeddings(embedding_config, mode='train'):
    print(f"\n⏳ Loading {mode} embeddings...")
    feature_list = []
    for name, paths in embedding_config.items():
        path = paths[mode]
        emb = np.load(path, mmap_mode='r')
        feature_list.append(emb)

    # Ghép nối
    X = np.concatenate([arr[:] for arr in feature_list], axis=1).astype(np.float32)
    print(f"   ✓ Raw Shape: {X.shape}")
    del feature_list, emb
    gc.collect()
    return X

# ============================================================================
# 2. PREPARE LABELS
# ============================================================================
print("\n[1/5] Processing Labels...")
train_ids = np.load(CONFIG['TRAIN_ID_PATH'])
df_terms = pd.read_csv(CONFIG['TRAIN_TERMS'], sep='\t', header=0, names=['EntryID', 'term', 'aspect'])
df_ia = pd.read_csv(CONFIG['IA_FILE'], sep='\t', names=['term', 'ia'])
ia_dict = dict(zip(df_ia['term'], df_ia['ia']))

term_counts = df_terms['term'].value_counts().reset_index()
term_counts.columns = ['term', 'freq']
term_counts['ia'] = term_counts['term'].map(ia_dict).fillna(0.0)
term_counts['score'] = term_counts['freq'] * term_counts['ia']

top_terms_df = term_counts.sort_values(by='score', ascending=False).head(CONFIG['TOP_K_LABELS'])
top_terms = top_terms_df['term'].tolist()
print(f"   ✓ Selected Top {len(top_terms)} terms (Reduced for VRAM safety).")

df_filtered = df_terms[df_terms['term'].isin(top_terms)]
id_to_terms = df_filtered.groupby('EntryID')['term'].apply(list).to_dict()

del df_terms, df_filtered, term_counts, top_terms_df
gc.collect()

print("   ⏳ Converting to Binary Matrix...")
mlb = MultiLabelBinarizer(classes=top_terms, sparse_output=True)
mlb.fit([top_terms])
y_labels_list = [id_to_terms.get(pid, []) for pid in train_ids]
Y_sparse = mlb.transform(y_labels_list)
Y = Y_sparse.astype(np.float32).toarray()

del y_labels_list, train_ids, Y_sparse
gc.collect()

model_path = f"{CONFIG['SAVE_DIR']}/pyboost_model.pkl"
print(f"\n🚀 Loading Model from: {model_path}")
model = joblib.load(model_path)

# ============================================================================
# 5. INFERENCE
# ============================================================================
print("\n[4/5] Inference on Test Set...")

# Load Test Features
X_test = load_embeddings(CONFIG['EMBEDDINGS'], mode='test')

test_ids = np.load(CONFIG['TEST_ID_PATH'])

print("   🔮 Predicting...")
preds = model.predict(X_test, batch_size=2048)

print("   📝 Writing Submission...")
submission_path = f"{CONFIG['SAVE_DIR']}/submission.tsv"
n_predictions = 0

with open(submission_path, 'w') as f:
    for i, pid in enumerate(tqdm(test_ids)):
        probs = preds[i]
        top_k = CONFIG['MAX_PREDS_PER_PROTEIN']
        ind = np.argpartition(probs, -top_k)[-top_k:]
        ind = ind[np.argsort(probs[ind])][::-1]

        for idx in ind:
            score = probs[idx]
            if score > CONFIG['MIN_CONFIDENCE']:
                term = top_terms[idx]
                f.write(f"{pid}\t{term}\t{score:.3f}\n")
                n_predictions += 1

print(f"\n✅ DONE! Predictions: {n_predictions:,}")

🚀 CAFA 6 - PY-BOOST LIGHTWEIGHT | GPU: 0

[1/5] Processing Labels...
   ✓ Selected Top 5000 terms (Reduced for VRAM safety).
   ⏳ Converting to Binary Matrix...

🚀 Loading Model from: /content/drive/MyDrive/CAFA6_Results/PyBoost_Run/pyboost_model.pkl

[4/5] Inference on Test Set...

⏳ Loading test embeddings...
   ✓ Raw Shape: (224309, 3072)
   🔮 Predicting...
   📝 Writing Submission...


  0%|          | 0/224309 [00:00<?, ?it/s]


✅ DONE! Predictions: 4,071,318


In [ ]:
# Chạy lệnh này trong một cell mới
!kaggle competitions submit \
    -c cafa-6-protein-function-prediction \
    -f /content/drive/MyDrive/CAFA6_Results/PyBoost_Run/submission.tsv \
    -m "esm2 ankh prot t5 pyboost top_k_labels=5k n_trees=10k lr=0.01 max_depth=6 es=300"

100% 93.5M/93.5M [00:01<00:00, 50.5MB/s]
Successfully submitted to CAFA 6 Protein Function Prediction

In [ ]:
!kaggle competitions submissions -c cafa-6-protein-function-prediction

fileName        date                        description                                                                             status                     publicScore  privateScore  
--------------  --------------------------  --------------------------------------------------------------------------------------  -------------------------  -----------  ------------  
submission.tsv  2025-12-16 09:26:33.940000  esm2 ankh prot t5 pyboost top_k_labels=5k n_trees=10k lr=0.01 max_depth=6 es=300        SubmissionStatus.COMPLETE  0.281                      
submission.tsv  2025-12-16 08:43:49.533000  esm2 ankh prot t5 pyboost top_k_labels=5k n_trees=10k lr=0.01 max_depth=6               SubmissionStatus.COMPLETE  0.250                      
submission.tsv  2025-12-15 14:15:13.093000  esm2 ankh prot t5 pyboost top_k_labels=5k n_trees=10k lr=0.01 max_depth=6 chunk_aspect  SubmissionStatus.COMPLETE  0.273                      
submission.tsv  2025-12-15 11:03:49.053000  esm2 ankh prot t5 pyb